# Snowflake as Data Source for training an ML Model with Amazon Sagemaker


This notebook works well with the `Python 3 (Data Science)` kernel on a SageMaker Studio Notebook `ml.t3.medium` instance.


---

## Connection to Snowflake Account

In [ ]:

import os
import boto3
import pandas as pd 
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

#Collect snowflake credentials from Secrets Manager
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='your secret ID from Secret Manager')
secrets_credentials = json.loads(response['SecretString'])
sf_password = secrets_credentials['password']

#Connect to Snowflake account
conn = snowflake.connector.connect(
    user        = "your_username",
    password    =  sf_password,
    account     = "your_accountid",
    warehouse   = "warehouse",
    protocol    = "https"
    )


---

## Snowflake Table Creation 

In [ ]:
# Connect to Snowflake database
snowflake_db = "snowflake-database-name"
conn.cursor().execute("USE DATABASE " + snowflake_db)

# Connect to Snowflake Table schema
housing_schema = 'snowflake-schema-name'
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS " + housing_schema)

conn.cursor().execute("USE SCHEMA " + housing_schema)

dataset_file = "housing.csv"

# Getting filename in UPPERCASE and removing .csv
filename = os.path.splitext(os.path.basename(dataset_file))[0].upper()

# Name for snowflake table
snowflake_table =  filename + 'TABLE'

# Create a CREATE TABLE SQL-statement
create_table_sql = "CREATE TABLE IF NOT EXISTS " + snowflake_db + "." + housing_schema + "." + snowflake_table + " (\n"

# Making a data frame to read the columns from 
df_housing = pd.read_csv(dataset_file) 
df_housing.rename(columns=str.upper, inplace=True)
df_housing.columns

# Iterating trough the columns
for col in df_housing.columns:
    column_name = col.upper()

    if (df_housing[col].dtype.name == "int" or df_housing[col].dtype.name == "int64"):
        create_table_sql = create_table_sql + column_name + " int"
    elif df_housing[col].dtype.name == "object":
        create_table_sql = create_table_sql + column_name + " varchar(16777216)"
    elif df_housing[col].dtype.name == "datetime64[ns]":
        create_table_sql = create_table_sql + column_name + " datetime"
    elif df_housing[col].dtype.name == "float64":
        create_table_sql = create_table_sql + column_name + " float8"
    elif df_housing[col].dtype.name == "bool":
        create_table_sql = create_table_sql + column_name + " boolean"
    else:
        create_table_sql = create_table_sql + column_name + " varchar(16777216)"

    # Deciding next steps. Either column is not the last column (add comma) else end create_tbl_statement
    if df_housing[col].name != df_housing.columns[-1]:
        create_table_sql = create_table_sql + ",\n"
    else:
        create_table_sql = create_table_sql + ")"

        #Execute the SQL statement to create the table
        conn.cursor().execute(create_table_sql)  

conn.cursor().execute('TRUNCATE TABLE IF EXISTS ' + snowflake_table)    

# Write the data from the DataFrame to the Snowflake table.
write_pandas(
        conn=conn,
        df=df_housing,
        table_name=snowflake_table,
        database=snowflake_db,
        schema=housing_schema
    ) 